In [ ]:
import cv2
import numpy as np
import math
import os
import fnmatch
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

In [ ]:
def blacked(img,ground,mask,d):
    mask[mask>127]=255

    ##Spliting Channels (0,1,2 represents B,G,R)
    G=img[:,:,1]

    # cv2.imshow("Original Green", G)
    # cv2.imshow("Ground", ground)

    ##Thresholding the channels
    G[ground>127]=0
    # cv2.imshow("Blacked Green", G)


    # cv2.waitKey(0)
    # cv2.destroyAllWindows()

    #Save the images
    # path = '/home/rayuga/Documents/Project_IEM/Outputs/blackedGreen'
    # cv2.imwrite(os.path.join(path,f"{d}.png"), G)
    return G

In [ ]:
def get_vessels(black_indices):
  result = []
  sub_list = []
  previous_element = None

  for element in black_indices:
    if previous_element is None or element == previous_element + 1:
      sub_list.append(element)
    else:
      result.append(sub_list)
      sub_list = [element]
    previous_element = element

  # Append the last sub-list to the result list if it is not empty.
  if len(sub_list)!=0:
    result.append(sub_list)

  return result

def generate_channel(c1,c2,mask,x,y):
    for e,row in enumerate(mask,0):
        white_indices = np.where(row == 255)[0]
        black_indices=[]
        if len(white_indices)!=0:
          for i in white_indices:
            if c1[e][i]==0:
                black_indices.append(i)
          if len(black_indices)!=0:
            vessels=get_vessels(black_indices)
            #print(e,vessels)
            for vessel in vessels:
                #creating 11x11 matrix on the left side
                matrix=[]
                new_matrix=[]
                col_index=vessel[0]-1
                for i in range(5):
                    for j in range(3):
                        row_index=abs(e-j)#(e-j) if j <= e else -1*(e-j)
                        matrix.append(c1[row_index][col_index])
                        if c1[row_index][col_index]>y:
                            new_matrix.append(c1[row_index][col_index])
                    for j in range(1,3):
                        row_index=e+j
                        matrix.append(c1[row_index][col_index])
                        if c1[row_index][col_index]>y:
                            new_matrix.append(c1[row_index][col_index])
                    col_index-=1

                if len(new_matrix)!=0:
                    p1=sum(new_matrix)//len(new_matrix)
                else:
                    p1=sum(matrix)//len(matrix) #problem
                
                #creating 11x11 matrix on the right side
                matrix=[]
                new_matrix=[]
                col_index=vessel[-1]+1 #len(vessel)
                for i in range(5):
                    for j in range(3):
                        row_index=abs(e-j)#(e-j) if j <= e else -1*(e-j)
                        matrix.append(c1[row_index][col_index])
                        if c1[row_index][col_index]>y:
                            new_matrix.append(c1[row_index][col_index])
                    for j in range(1,3):
                        row_index=e+j
                        matrix.append(c1[row_index][col_index])
                        if c1[row_index][col_index]>y:
                            new_matrix.append(c1[row_index][col_index])
                    col_index+=1
                    
                if len(new_matrix)!=0:
                    p2=sum(new_matrix)//len(new_matrix)
                else:
                    p2=sum(matrix)//len(matrix) #problem
                
                n=len(vessel)

                b=[]

                for i in range(n):
                    res=((math.comb((n-1),i))*(x**i)*((1-x)**((n-1)-i)))
                    b.append(res)

                diff=abs(p1-p2)
                index=0
                for i in b:
                    c=diff*i
                    if p1>p2:
                        c2[e][vessel[index]]=p1-c
                    else:
                        c2[e][vessel[index]]=p1+c
                    p1=c2[e][vessel[index]]
                    index+=1
            
    return c2

def merge_channel(c1,c2,c,row,col):
    for i in range(row):
        for j in range(col):
            c[i][j]=max(c2[i][j],c1[i][j])

In [ ]:
# def get_vessels(black_indices):
#   result = []
#   sub_list = []
#   previous_element = None

#   for element in black_indices:
#     if previous_element is None or element == previous_element + 1:
#       sub_list.append(element)
#     else:
#       result.append(sub_list)
#       sub_list = [element]
#     previous_element = element

#   # Append the last sub-list to the result list if it is not empty.
#   if len(sub_list)!=0:
#     result.append(sub_list)

#   return result

# def generate_channel(c1,c2,mask,x,y):
#     for e,row in enumerate(mask,0):
#         white_indices = np.where(row == 255)[0]
#         black_indices=[]
#         if len(white_indices)!=0:
#           for i in white_indices:
#             if c1[e][i]==0:
#                 black_indices.append(i)
#           if len(black_indices)!=0:
#             vessels=get_vessels(black_indices)
#             for vessel in vessels:
#                 #creating 11x11 matrix on the left side
#                 matrix=[]
#                 new_matrix=[]
#                 col_index=vessel[0]-1
#                 for i in range(7):
#                     for j in range(4):
#                         row_index=abs(e-j)#(e-j) if j <= e else -1*(e-j)
#                         matrix.append(c1[row_index][col_index])
#                     for j in range(1,4):
#                         row_index=e+j
#                         matrix.append(c1[row_index][col_index])
#                     col_index-=1

#                 p1=sum(matrix)//len(matrix)
                
#                 #creating 11x11 matrix on the right side
#                 matrix=[]
#                 new_matrix=[]
#                 col_index=vessel[-1]+1 #len(vessel)
#                 for i in range(7):
#                     for j in range(4):
#                         row_index=abs(e-j)#(e-j) if j <= e else -1*(e-j)
#                         matrix.append(c1[row_index][col_index])
#                     for j in range(1,4):
#                         row_index=e+j
#                         matrix.append(c1[row_index][col_index])
#                     col_index+=1
                    
#                 p2=sum(matrix)//len(matrix)
                
#                 n=len(vessel)

#                 b=[]

#                 for i in range(n):
#                     res=((math.comb((n-1),i))*(x**i)*((1-x)**((n-1)-i)))
#                     b.append(res)

#                 diff=abs(p1-p2)
#                 index=0
#                 for i in b:
#                     c=diff*i
#                     if p1>p2:
#                         c2[e][vessel[index]]=p1-c
#                     else:
#                         c2[e][vessel[index]]=p1+c
#                     p1=c2[e][vessel[index]]
#                     index+=1
            
#     return c2

# def merge_channel(c1,c2,c,row,col):
#     for i in range(row):
#         for j in range(col):
#             c[i][j]=max(c2[i][j],c1[i][j])

In [ ]:
def fillVes(G,mask):
    G=cv2.copyMakeBorder(G, 5, 5, 5, 5, cv2.BORDER_CONSTANT,0)
    mask=cv2.copyMakeBorder(mask, 5, 5, 5, 5, cv2.BORDER_CONSTANT,0)
    #Green channel
    #horizontal
    g_h2=generate_channel(G.copy(),G.copy(),mask,0.2,50)

    #vertical
    g_v2=generate_channel(G.copy().T,G.copy().T,mask.T,1,50)
    g_v2=g_v2.T

    #get row and col
    shape=img.shape
    row=shape[0]
    col=shape[1]

    #create new green channel
    g=np.empty(shape=[row,col],dtype=np.uint8)
    merge_channel(g_h2,g_v2,g,row,col)

    # Save the images
    path = '/home/rayuga/Documents/Project_IEM/Outputs/filledGreen2'
    cv2.imwrite(os.path.join(path,f"{d}.png"), g)
    print('saved')

In [ ]:
#feature extraction
def extractFeature(G):
    features = []

    for i,row in enumerate(mask):
        white_indices = np.where(row == 255)[0]
        if len(white_indices)!=0:
            for col in white_indices:
                sub = []
                for x in range(i - 1, i + 2):
                    for y in range(col - 1, col + 2):
                        if not(x==i and y==col):
                            substract = abs(int(G[x][y]) - int(G[i][col]))
                            sub.append(substract)

                minimum = min(sub)
                features.append([G[i][col], minimum])
    return features

def gen_Regions(matrix,mask):
    features = extractFeature(matrix)

    # K-means clustering

    kmeans = KMeans(n_clusters=3,init="k-means++",n_init="auto",random_state=0)

    kmeans.fit_predict(features)

    labels = kmeans.labels_
    #print(np.unique(labels))

    # Create a scatter plot to visualize the clusters
    # plt.figure(figsize=(19,6))
    # plt.scatter(*zip(*features), c=labels, cmap='viridis')

    # plt.title('K-Means Clustering')
    # plt.show()

    #get row and col
    shape=matrix.shape
    row=shape[0]
    col=shape[1]

    k0_img=np.zeros((row,col),dtype=np.uint8)
    k1_img=np.zeros((row,col),dtype=np.uint8)
    k2_img=np.zeros((row,col),dtype=np.uint8)

    #region creation
    for i,row in enumerate(mask):
        white_indices = np.where(row == 255)[0]
        if len(white_indices)!=0:
            for col in white_indices:
                sub = []
                for x in range(i - 1, i + 2):
                    for y in range(col - 1, col + 2):
                        if not(x==i and y==col):
                            substract = abs(int(matrix[x][y]) - int(matrix[i][col]))
                            sub.append(substract)

                minimum = min(sub)
                f_index=features.index([matrix[i][col],minimum])
                #print(labels[f_index])
                match labels[f_index]:
                    case 0:
                        k0_img[i][col]=255
                        #print("kluster 1")
                    case 1:
                        k1_img[i][col]=255
                        #print("kluster 2")
            
                    case 2:
                        k2_img[i][col]=255
                        #print("kluster 3")
            
                    case _:
                        print("Error!!!")

    #Save the images

    path = f'/home/rayuga/Documents/Project_IEM/Outputs/regions2/{d}'
    os.mkdir(path)

    cv2.imwrite(os.path.join(path,"k0.png"), k0_img)
    cv2.imwrite(os.path.join(path,"k1.png"), k1_img)
    cv2.imwrite(os.path.join(path,"k2.png"), k2_img)

In [ ]:
for (root,dirs,_) in os.walk("/home/rayuga/Documents/Project_IEM/Outputs/image_DB"):
    for d in dirs:
        print(d)
        for (_,_,files) in os.walk(root+'/'+d):
            for f in files:
                print(f)
                if fnmatch.fnmatch(f,'*_man*'):
                    ground = cv2.imread(root+'/'+d+'/'+f,0)
                elif fnmatch.fnmatch(f,'*_train*') and f.endswith('.jpg'):
                    mask = cv2.imread(root+'/'+d+'/'+f,0)
                elif f.endswith('.tif'):
                    img = cv2.imread(root+'/'+d+'/'+f)
   
            G=blacked(img,ground,mask,d)
            #G[G<60]=0
            fillVes(G,mask)

            matrix=cv2.imread(f"/home/rayuga/Documents/Project_IEM/Outputs/filledGreen2/{d}.png",0)
            gen_Regions(matrix,mask)

In [ ]:
# def erode(source,destination,kernel):
#     for (root,_,files) in os.walk(source):
#         for f in files:
#             image = cv2.imread(root+f)
            
#             # Using cv2.erode() method  
#             image = cv2.erode(image, kernel,iterations=30)

#             if not os.path.exists(destination):
#                 os.mkdir(destination)
#             cv2.imwrite(os.path.join(destination,f), image)

# source='/home/rayuga/Documents/Project_IEM/Outputs/filledGreen2/'
# destination='/home/rayuga/Documents/Project_IEM/Outputs/eroded/'
# # Creating kernel 
# kernel = np.random.randint(low=0, high=2, size=(4,4), dtype=np.uint8)
# print(kernel)
# erode(source,destination,kernel)
